In [34]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

In [64]:
M1,m2 = 0.2 , 0.034
mean_interval,skew_interval,mean_overlap,skew_overlap = m2,0.14,m2/4,0.035
var,var_no  =0.2,0.1 #variance for yes and no gaussian
pos_th, en_th = 0.4, 11
skew_change,mean_change = (skew_interval+2*skew_overlap)/10, (m2+2*mean_overlap)/10 #Change in mean and skew during a decision
mn_max, sk_max = M1, 0 #initial values of mean and skew for apathy = 0
w = 0.0005 #weighting on updating values in the learning process just /10
en_max = 1.5
neg_th = 0.7




In [36]:

#Defines the intervals the mean and skew are bound by when a decision is being made 
def interval(mean_interval,skew_interval,mean_overlap,skew_overlap,mn,sk,ap):
    mean_min = mn-mean_interval-mean_overlap
    mean_max = mn+mean_interval+mean_overlap
    skew_min = sk-skew_interval-skew_overlap
    skew_max = sk+skew_interval+skew_overlap
    
    return[mean_min,mean_max,skew_min,skew_max]
    

In [37]:
#Defines how the mean and skew will change during a decision, bound by the interval function 
def deliberation_no(mean_ap,skew_ap,intervals):
    skew = skew_ap-skew_change
    mean = mean_ap - mean_change

    mins = [intervals[2],intervals[0]]
    if skew< mins[0]:
        skew = mins[0]
    if mean < mins[1]:
        mean=mins[1]
    return [mean,skew]
    
def deliberation_yes(mean_ap,skew_ap,intervals):
    skew = skew_ap + skew_change
    mean = mean_ap + mean_change

    maxs = [intervals[3],intervals[1]]
    if skew>maxs[0]:
        skew = maxs[0]
    if mean > maxs[1]:
        mean = maxs[1]
    return [mean,skew]

In [38]:
#Defines the initial mean and skew for a given apathy to be used at the start of a simulation
def initial(ap,mean_interval,skew_interval,mean_overlap,skew_overlap):
    decimal = ap%1
    ap = int(ap)
    if ap>10 or ap<0:
        return 'error'

    
    if ap !=0 and ap!=9 and ap!=10:
        mn = mn_max +mean_overlap-(ap)*mean_interval - (mean_interval+2*mean_overlap)*decimal
        sk = sk_max +  skew_overlap-(ap)*skew_interval - (skew_interval+2*mean_overlap)*decimal
    
    elif ap==9:
        mn = mn_max - 10*mean_interval+(mean_interval+mean_overlap)*(1-decimal) 
        sk = sk_max-10*skew_interval + (skew_interval+skew_overlap)*(1-decimal)
        
    elif ap==0:
        mn = mn_max - (mean_interval+mean_overlap)*decimal
        sk = sk_max - (mean_interval+mean_overlap)*decimal

    elif ap==10:
        mn = mn_max - 10*mean_interval
        sk = sk_max-10*skew_interval
       
    return [mn,sk]



In [ ]:
#Energy used per deliberation, or 0.25
def en_ap(ap):
    return (0.1+ap*0.25)

In [40]:
#The process of making a decision to engage or not by sampling under the gaussian
# en is the energy usage
#while loop ensures no value greater than 1 
def decision_process(mean_ap,skew_ap,ap):
  yes = no = 99
  while yes>1 or no>1:
    yes = stats.skewnorm.rvs(skew_ap,mean_ap,var)
    no = np.random.normal(0,var_no)
      
    flg=0

    val = abs(yes-no)
    en = en_ap(ap)/val
      
    if en >en_max:
      en = en_max
    if yes>no:
      flg=1

    return [flg,val,en,yes,no]

In [41]:
#one entire decision
def one_decision(mn,sk,ap):
    
    rep = 0 #number of processes in the decision

    pos = 0 #value assigned to engaging, if it reaches pos_th the decision is engage
    neg = 0 #value of not engaging, if it reaches neg_th the decision is not engage
    en = 0  #value of energy used, if it reaches en_th the decision is not engage

    yes = 0 #total engage value sampled when it is greater than not engage
    no = 0  #total not engage value sampled when it is greater than engage
    ys = 0  #total times engage was greater
    n = 0   #total times not engage was greater

    en_flg = 0 #indicates if en reached en_th
    want = 0 #if exhaustion is reached, but the subject wanted to engage, this has a value

    mn_dec = mn #Can change during a decision
    sk_dec = sk #Can change during a decision
    intervals = interval(mean_interval,skew_interval,mean_overlap,skew_overlap,mn,sk,ap)

    while pos < pos_th and neg < neg_th and en<en_th:
        
        deliberation = decision_process(mn_dec,sk_dec,ap)

        if deliberation[0] == 1:   #engage
            pos_change = deliberation_yes(mn_dec,sk_dec,intervals)
            mn_dec = pos_change[0]
            sk_dec = pos_change[1]
            pos +=deliberation[1]
            yes+=deliberation[3]
            ys+=1 

        else: #disengage
            neg_change = deliberation_no(mn_dec,sk_dec,intervals)
            mn_dec = neg_change[0]
            sk_dec = neg_change[1]
            neg +=deliberation[1]
            no+=deliberation[4]
            n+=1

        en+=deliberation[2]
        rep+=1
    
    if en>en_th and pos<pos_th and neg<neg_th:
        en_flg = 1
        if pos>neg:
            want = yes/ys
    if pos<pos_th:
        if n==0:
            n = 1
            no += deliberation[4]
            print('fringe case occured, run again')
        
    return [1,en,yes/ys,rep,en_flg,want] if pos>pos_th else [0,en,no/n,rep,en_flg,want]

In [42]:
#one entire decision
#define en and pos before calling
def one_decision_CBT(mn,sk,ap,en,pos):
    
    rep = 0 #number of processes in the decision

    
    neg = 0 #value of not engaging, if it reaches neg_th the decision is not engage
    

    yes = 0 #total engage value sampled when it is greater than not engage
   
    ys = 0  #total times engage was greater
    n = 0   #total times not engage was greater

    en_flg = 0 #indicates if en reached en_th
    want = 0 #if exhaustion is reached, but the subject wanted to engage, this has a value

    mn_dec = mn #Can change during a decision
    sk_dec = sk #Can change during a decision
    intervals = interval(mean_interval,skew_interval,mean_overlap,skew_overlap,mn,sk,ap)

    while pos < pos_th and neg < neg_th and en<en_th:
        
        deliberation = decision_process(mn_dec,sk_dec,ap)

        if deliberation[0] == 1:   #engage
            pos_change = deliberation_yes(mn_dec,sk_dec,intervals)
            mn_dec = pos_change[0]
            sk_dec = pos_change[1]
            pos +=deliberation[1]
            yes+=deliberation[3]
            ys+=1 

        else: #disengage
            neg_change = deliberation_no(mn_dec,sk_dec,intervals)
            mn_dec = neg_change[0]
            sk_dec = neg_change[1]
            neg +=deliberation[1]
            
            n+=1

        en+=deliberation[2]
        rep+=1
    
    if en>en_th and pos<pos_th and neg<neg_th:
        en_flg = 1
        if pos>neg and ys!=0:
            want = yes/ys
    
        
    return [1,en,yes,ys,rep,en_flg,want] if pos>pos_th else [0,en,yes,ys,rep,en_flg,want]

In [63]:
#Energy available in a day
def day_en_th_fun(x):
    return (-176/117*x**2+1058/117*x+100)*1.5/0.8

In [54]:
#All decisions across d days
def day_decisions(ap , days,f_mn,f_sk,f_ap,f_ac,f_en,f_CBT,f_ADM,f_mood,f_disengage,f_CBT_2):

    dep = ap/10*0.6 # defines an initial depression score because mood is only generated at the start of the day using dep
    
     
    
    #All lists generated for perturbing the model
    day_m1,del_m1 = create_lists(f_mn)
    day_sk,del_sk = create_lists(f_sk)
    day_ap,del_ap = create_lists(f_ap)
    day_actual,actual_pos = create_lists(f_ac)
    day_energy = create_lists_en(f_en)
    day_CBT = create_lists_en(f_CBT)
    day_ADM = create_lists_en(f_ADM)
    day_mood,mood_cap = create_lists(f_mood)
    day_disengage = create_lists_en(f_disengage)
    day_CBT_2 = create_lists_en(f_CBT_2)

    start = initial(ap,mean_interval,skew_interval,mean_overlap,skew_overlap)
    sk = start[1]
    
    
    m1=M1 #allows m1 to change
    
    no = 0 #total number of not engage decisions
    ys = 0 #total engage decisions made across all days
    flg = 1 #flag if the current decisions is engage, also used to indicate if the previous decision engaged
    en_flg = 0 #total number of exhausts from one decision in a day
    day_exhaust = 0 #number of complete exhaustions for a day
    
    #evolution over all days
    ev_mn=[]
    ev_sk=[]
    ev_ap =[]
    ev_pos_dec = []
    ev_neg_dec = []
    ev_postoneg = []
    ev_rep = []
    ev_exhaust = []
    ev_day_exhaust = []
    ev_mood = []
    ev_dep = []

    for d in range(days):

        mood = -1
        while mood<0 or mood >1:
            mood =  np.random.normal(0.9-dep*0.8,0.1)
        md_cap = force_mood(day_mood,mood_cap,d) #forcing a period of low mood
        if mood > md_cap: #Cap is 1 when we dont force mood
                mood = md_cap
        
       

        
        sk = ADM(day_ADM, 50*w, d, sk) #ADM changing the skew, the learning process will deal with this once ADM is stopped
        
        day_en = 0 #energy used in  a day
        dec = 0 # max of 10 decisions in a day
        yes = 0  #total engage decisions in a day
        day_en_th = day_en_th_fun(ap) 

        day_en+=force_energy(day_energy,d) #Can add a daily energy usage to force exhaustion
        sk+=force_skew(day_sk,del_sk,d) #can force a change in skew
        ap+=force_ap(day_ap,del_ap,d) #can force a change in ap
        m1+=force_mean(day_m1,del_m1,d) # can fore a change in m1 hence mean
        disengage = force_disengage(day_disengage,d) #Force disengagement for a period
        cbt_2 = CBT_2(day_CBT_2,d)

        while dec<10 and day_en<day_en_th:

            mn = mean(dep*10,m1,m2)
            if cbt_2 ==1:
                
                en,pos,outcome,exhaust,loop = 0,0,0,0,0
                total_yes=0
                total_yes_decs=0
                rep = 0
                while outcome ==0 and exhaust ==0:
                    decision = one_decision_CBT(mn,sk, ap,en,loop*preference)
                    outcome = decision[0]
                    en+=decision[1]
                    total_yes+=decision[2]
                    total_yes_decs+=decision[3]
                    rep+=decision[4]
                    exhaust = decision[5]
                    loop+=1
                predicted = 0
                if total_yes_decs!=0:
                    predicted = total_yes/total_yes_decs
                want = 0
                if decision[6]!=0:
                    want = predicted

                if outcome==1 and flg ==1:
                    flg_con = 1 # two engages in a row
                else:
                    flg_con = 0
                flg = outcome #1 for engage
                en_flg+=exhaust  #1 for exhaustion
                day_en+=en #energy used by decision
                rep = rep # deliberations of a decisions
                dec+=1
                if flg ==1:
                    ys+=1
                    yes+=1
                elif flg==0:
                    no+=1

                learn = param_learn(ap,m1,m2,predicted,flg,flg_con,sk,d,day_actual,actual_pos,dep*10,exhaust,want)

            elif  disengage ==0: #not forcing disengage
                decision = one_decision(mn,sk, ap)
                if decision[0]==1 and flg ==1:
                    flg_con = 1 # two engages in a row
                else:
                    flg_con = 0

                flg = decision[0] #1 for engage
                en_flg+=decision[4]  #1 for exhaustion
                day_en+=decision[1] #energy used by decision
                rep=decision[3] # deliberations of a decisions
                dec+=1

                if flg ==1:
                    ys+=1
                    yes+=1
                elif flg==0:
                    no+=1

                learn = param_learn(ap,m1,m2,decision[2],flg,flg_con,sk,d,day_actual,actual_pos,dep*10,decision[4],decision[5]) # learning process to update m1 (hence mean) skew and apathy
            
            else: #forcing disengage
                
                flg_con = 0
                flg = 0
                rep = 0
                dec+=1
                no+=1
                learn = param_learn(ap,m1,m2,0,flg,flg_con,sk,d,day_actual,actual_pos,dep*10,0,0)
            
            m1 = learn[0]
            
            sk+=learn[1]
            ap+=learn[2]
            mood+=learn[3]/10
            
            if mood<0:
                mood = 0
            if mood > md_cap: #Cap is 1 when we dont force mood
                mood = md_cap
            
            if ap<0:
                ap = 0
            if ap>10:
                ap = 10
            
            ev_mood.append(mood)
            ev_mn.append(mn)
            ev_sk.append(sk)
            ev_ap.append(ap)
            ev_pos_dec.append(ys)
            ev_neg_dec.append(no)
            ev_rep.append(rep)
            ev_exhaust.append(en_flg)
            
            day_en_th = day_en_th_fun(ap) #update daily en_th for new ap
           
        sk+=learn_sk(yes) #update the skew after each day with the number of engages made in a day

        if day_en>=day_en_th: #Ran out of energy in a day

            flg = 0
            flg_con = 0
            day_exhaust+=1

            for i in range(10-dec):
                #Updating still occurs during complete exhaustion, just the updating of not engaging
                mn = mean(dep*10,m1,m2)

                learn = param_learn(ap,m1,m2,0,flg,flg_con,sk,d,day_actual,actual_pos,dep*10,1,0) # learning process to update m1 (hence mean) skew and apathy
                m1 = learn[0]
                sk+=learn[1]
                ap+=learn[2]
                
                if ap<0:
                    ap = 0
                if ap>10:
                    ap = 10
               
                mood+=learn[3]/10

                if mood > md_cap: #Cap is 1 when we dont force mood
                    mood = md_cap
                if mood<0:
                    mood = 0

                en_flg+=1 #still counts as a decision made by exhaustion
                no+=1

                ev_mn.append(mn)
                ev_sk.append(sk)
                ev_ap.append(ap)
                ev_pos_dec.append(ys)
                ev_neg_dec.append(no)
                ev_exhaust.append(en_flg)
                ev_mood.append(mood)

        dep = ap/10*(1-0.7)+(1-mood)*0.7  #Dep depends more on low mood than apathy

        ev_dep.append(dep)
        ev_day_exhaust.append(day_exhaust)
        
    return [ev_mn,ev_sk,ev_ap,ev_rep,ev_pos_dec,ev_neg_dec, ev_exhaust,ev_day_exhaust,ev_mood,ev_dep]


In [45]:

def mean(ap,m1,m2):
    mu = m1-ap*m2
    return mu

def reward_sens(ap):
    return (1-ap/20)

def param_learn(ap,m1,m2,val,flg,flg_con,skew,d,day_actual,actual_pos,dep,exhaust,want): #Do I want different learing for exhaustion, should the mean/skew really be updated, apathy should
    
    del_sk = 0
    m1_new = m1

    actual = force_actual(day_actual,actual_pos,d) #Forces a value for the actual outcome
    if actual == -99: #vaue when we do not force an actual outcome
        actual = np.random.normal(M1,var)*reward_sens(ap)
    

    if flg==1: #engage
        dif = (actual-val) #actual*reward sensitivity - predicted
        m1_new = m1+w*dif
        if dif<0: 
            del_sk = -1/10*w*36 #if engage is worse than expected skew negatively
        
            
        del_mood = actual
    elif exhaust ==1:
        if want!=0:
            del_mood = np.random.normal(0,var_no) - want
        mn = mean(dep,m1,m2)
        m, vr, sk, kurt = stats.skewnorm.stats(skew,mn,var, moments='mvsk')
        
        del_mn = mn - (mn*vr**2)/(var_no**2+vr**2) # multiplication of current and not engage gaussians, without updating var, modified Bayesian Inference
        m1_new = m1 - del_mn*w*0.1
        del_mood = np.random.normal(0,var_no/2)
        if del_mood<=0:
            del_sk = 1/10*w*30

            

    elif flg==0:
        mn = mean(dep,m1,m2)
        m, vr, sk, kurt = stats.skewnorm.stats(skew,mn,var, moments='mvsk')
        
        del_mn = mn - (mn*vr**2)/(var_no**2+vr**2) # multiplication of current and not engage gaussians, without updating var, modified Bayesian Inference
        m1_new = m1 - del_mn*w
        del_mood = np.random.normal(0,var_no)

        if del_mood<=0:
            del_sk = 1/10*w*30 #Creates a balance between skew and mean
        
    #If two consecutive engages apathy improves (towards 0), if currently not engaging apathy gets worse, else no change
    del_ap = -w*40
    if flg_con ==0:
        del_ap*=-1 
        if flg == 1:
            del_ap = 0
    

    
    return(m1_new,del_sk,del_ap,del_mood)

def learn_sk(engage):
    del_sk = (engage-5)/10*w*40
    return(del_sk)




In [46]:
#Functions to alter the course of the model can be seen as inducing low mood or treating a patient
#day will be an array of days when alterations occur
#del_* will be the changes which occir on that day
#Sequential days will have the same del_*
#brakes in sequential days will have value -1, this moves on to the next del_* value

def force_mean(day:[],del_m1:[],d):
    if day:
        if d ==day[-1]:
            day.pop()
            return del_m1[-1]
            
        elif day[-1]==-1:
            del_m1.pop()
            day.pop()
        
    return 0 




def force_skew(day:[],del_sk:[],d):
    if day:
        if d ==day[-1]:
            day.pop()
            if ADM == True and s >=0:
                return 0
            return del_sk[-1]
            
        elif day[-1]==-1:
            del_sk.pop()
            day.pop()
        
    return 0 



def force_actual(day:[],actual_pos:[],d): #forces a  value for actual positivity of event after engaging

    if day:
        if d ==day[-1]:
            day.pop()
            return actual_pos[-1]
            
        elif day[-1]==-1:
            actual_pos.pop()
            day.pop()
        
    return -99



def force_energy(day:[],d):  #forces high energy usage to force not engage without affecting distributions
    if day:
        if d ==day[-1]:
            day.pop()
            return 1000
        
    return 0 



def force_ap(day:[],del_ap:[],d):

    if day:
        if d ==day[-1]:
            day.pop()
            return del_ap[-1]
            
        elif day[-1]==-1:
            del_ap.pop()
            day.pop()
        
    return 0 

def force_mood(day:[],mood:[],d): #Force a maximum mood value for a period of days
    if day:
        if d ==day[-1]:
            day.pop()
            return mood[-1]
            
        elif day[-1]==-1:
            mood.pop()
            day.pop()
        
    return 1



def force_disengage(day:[],d): #Force disengagement for a period of days, set values (deliberation) as NaN and use numpy nanmean, force disengage or loop until disengage is chosen
    if day:
        if d ==day[-1]:
            day.pop()
            return 1
        
    return 0 





In [47]:
#f is in the format [day,duration,val,day,duration,val...]
#day the tool is active, number of days the tool is active, value which the tool changes by
# returns an array of active days and their active values

def create_lists(f):
    if f is None:
        return [-2],[-2]
    days = []
    dels = []
    
    for i in range(int(len(f)/3)):
        start = f[i*3]
        duration = f[i*3+1]+1
        val=f[i*3+2]
        dur = np.arange(start,start+duration,1)
        days.extend(dur)
        days.append(-1)
        dels.append(val)
    days.reverse()
    dels.reverse()
    return days, dels


def create_lists_en(f):
    if f is None:
        return [-2]
    days = []
    for i in range(int(len(f)/2)):
        start = f[i*2]
        duration = f[i*2+1]+1
        dur = np.arange(start,start+duration,1)
        days.extend(dur)
    days.reverse()
    return days



In [48]:
#takes the starting apathy, simulton length, days we want to perturb the system and the values they will be perturbed by 

def Simulation_driver(ap, days,f_mn=None,f_sk=None,f_ap=None,f_ac=None,f_en=None,f_ADM=None,f_CBT=None,f_mood=None,f_disengage=None, f_CBT_2=None):
    plt.rcParams["figure.figsize"] = (30, 16)
    
    fig, axs = plt.subplots(5, 2)
    x = np.arange(0,days*10)
    x2 = np.arange(0,days)
    
    for ax in axs.flat:
        ax.set(xlabel='decisions', ylabel='value')
        ax.ticklabel_format(useOffset=False)
    axs[3][1].set(xlabel='days', ylabel='value')
    axs[4][1].set(xlabel='days', ylabel='value')
    axs[4][0].set(xlabel='days', ylabel='value')

    out = day_decisions(ap , days, f_mn,f_sk,f_ap,f_ac,f_en,f_CBT,f_ADM,f_mood,f_disengage, f_CBT_2)
    
    y1 = out[0]
    y2 = out[1]
    y3 = out[2]
    y4 = out[3]
    x4 = np.arange(0,len(y4))
    y5 = out[4]
    y6 = out[5]
    y7 = out[6]
    y8 = out[7]
    y9 = np.asarray(out[8])
    y9_day = np.mean(y9.reshape(-1, 10), axis=1)
    y10 = out[9]
    print(np.average(y4))
    axs[0, 0].plot(x, y1)
    axs[0, 0].set_title('evolution of mean')
    axs[0, 1].plot(x, y2)
    axs[0, 1].set_title('evolution of skew')
    axs[1, 0].plot(x, y3)
    axs[1, 0].set_title('evolution of apathy')
    axs[1, 1].plot(x4, y4)
    axs[1, 1].set_title('evolution of deliberation for each decision')
    axs[2, 0].plot(x, y5)
    axs[2, 0].set_title('evolution of decisions to engage')
    axs[2, 1].plot(x, y6)
    axs[2, 1].set_title('evolution of decisions to not engage')
    axs[3, 0].plot(x, y7)
    axs[3, 0].set_title('evolution of exhaustions reached for a decision')
    axs[3, 1].plot(x2, y8)
    axs[3, 1].set_title('evolution of exhaustions reached for a day')
    #axs[4, 0].plot(x, y9)
    #axs[4, 0].set_title('evolution of Mood')
    axs[4,0].plot(x2, y9_day)
    axs[4, 0].set_title('evolution of Mood')
    axs[4, 1].plot(x2, y10)
    axs[4, 1].set_title('evolution of MDD score')
    

    fig.tight_layout(pad=5.0)
    fig.suptitle('Starting Apathy of {}'.format(ap))
    plt.show()


In [49]:
#Day ADM starts on, duration of ADM, value the skew changes by (this might get fixed with research)
def ADM(day:[], val ,d , s):
    if day:
        if d ==day[-1]:
            day.pop()
            if s>=0.43:
                return s
            
            return s+val
            
        elif day[-1]==-1:
            del_sk.pop()
            day.pop()
    return s
    #Might want something to affect reward sensitivity
    #Or have different options depending on the drug of choice
    #Could model ADM as a boost to the intial prefrence state, this could wrk with repeated decision making process until we engge



def CBT_2(day:[],d):
    if day:
        if d ==day[-1]:
            day.pop()
            return 1
        elif day[-1]==-1:
                day.pop()
    return 0